In [1]:
""" Using convolutional net on MNIST dataset of handwritten digit
(http://yann.lecun.com/exdb/mnist/)
"""
from __future__ import print_function

import os
import time 

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

N_CLASSES = 10

In [2]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [69]:
# Step 2: Define paramaters for the model
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 50

In [111]:
tf.reset_default_graph()
# Step 3: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# We'll be doing dropout for hidden layer so we'll need a placeholder
# for the dropout probability too
# Use None for shape so we can change the batch_size once we've built the graph
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout')

In [112]:
# Step 4 + 5: create weights + do inference
# the model is conv -> relu -> pool -> conv -> relu -> pool -> fully connected -> softmax

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

In [113]:

with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    # use the dynamic dimension -1
    X_reshaped = tf.reshape(X, [-1, 28, 28, 1], name="X_reshaped")

    # create kernel variable of dimension [5, 5, 1, 32]
    # use tf.truncated_normal_initializer()
    W = tf.get_variable('weights', [5, 5, 1, 32], initializer=tf.truncated_normal_initializer())

    # create biases variable of dimension [32]
    # use tf.random_normal_initializer()
    b = tf.get_variable('biases', [32], initializer=tf.random_normal_initializer())
    
    # apply tf.nn.conv2d. strides [1, 1, 1, 1], padding is 'SAME'
    conv = tf.nn.conv2d(X_reshaped, W, strides=[1, 1, 1, 1], padding="SAME")
    
    # apply relu on the sum of convolution output and biases
    conv2 = tf.nn.relu(conv + b, name='conv1')
    

In [114]:
with tf.variable_scope('pool1') as scope:
    # apply max pool with ksize [1, 2, 2, 1], and strides [1, 2, 2, 1], padding 'SAME'
    pooled = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pooled')
    

In [115]:
with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    kernel = tf.get_variable('kernels', [5, 5, 32, 64], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [64],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pooled, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 14 x 14 x 64

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='SAME')

    # output is of dimension BATCH_SIZE x 7 x 7 x 64


In [116]:
with tf.variable_scope('fc') as scope:
    # use weight of dimension 7 * 7 * 64 x 1024
    input_features = 7 * 7 * 64
    
    # create weights and biases
    W_fc = tf.get_variable('weights_fc', [input_features, 1024])
    b_fc = tf.get_variable('biases_fc', [BATCH_SIZE, 1024])

    # reshape pool2 to 2 dimensional
    pool2 = tf.reshape(pool2, [-1, input_features])

    # apply relu on matmul of pool2 and w + b
    fc = tf.nn.relu(tf.matmul(pool2, W_fc) + b_fc)
    
    # apply dropout
    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')

In [117]:
with tf.variable_scope('softmax_linear') as scope:
    # this you should know. get logits without softmax
    # you need to create weights and biases
    W_s = tf.get_variable('W_s', [1024, 10])
    b_s = tf.get_variable('b_s', [1, 10])
    out = tf.matmul(fc, W_s)+ b_s


In [118]:
# Step 6: define loss function
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
with tf.name_scope('loss'):
    # you should know how to do this too
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, Y))



In [119]:
# Step 7: define training op
# using gradient descent with learning rate of LEARNING_RATE to minimize cost
# don't forgot to pass in global_step
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=global_step)


In [ ]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./my_graph/mnist', sess.graph)
    ##### You have to create folders to store checkpoints
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/convnet_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    initial_step = global_step.eval()

    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)

    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([optimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        total_loss += loss_batch
        print(loss_batch)
        if (index + 1) % SKIP_STEP == 0:
            print("Average loss at step {}: {:5.1f}".format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, './checkpoints/convnet_mnist/mnist-convnet', index)
    
    print("Optimization Finished!") # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    # test the model
    n_batches = int(mnist.test.num_examples/BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], 
                                        feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   
    
    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))

43.1864
134.14
185.316
153.681
156.261
60.9879
27.6666
15.5446
6.64192
2.53175
Average loss at step 10:  78.6
1.85028
1.93998
1.90275
1.63934
1.39433
1.61144
1.37294
1.88616
1.33188
1.52716
Average loss at step 20:   1.6
1.35512
1.30433
1.28596
1.24391
1.14077
1.27149
1.38269
1.09726
1.0485
1.15405
Average loss at step 30:   1.2
1.15828
0.921469
0.882207
1.01269
1.2039
1.24498
0.976512
0.889433
1.04719
0.870906
Average loss at step 40:   1.0
0.780757
0.77606
1.09103
1.14549
1.09111
1.00879
0.839388
0.772447
0.772932
1.06684
Average loss at step 50:   0.9
0.896091
0.704871
0.517724
0.51573
0.738422
0.672576
0.738668
0.73938
0.511591
0.621581
Average loss at step 60:   0.7
0.77831
0.695029
0.547368
0.744435
0.363315
0.502201
0.612616
0.93011
0.888483
0.587373
Average loss at step 70:   0.7
0.519983
0.514459
0.599801
0.62059
0.797866
0.594768
0.463431
0.667391
0.582274
0.51969
Average loss at step 80:   0.6
0.609779
0.481946
0.466194
1.01578
0.732936
0.473403
0.739276
0.716617
0.655971
0.